# 📘 代理架构 17：反思性元认知

欢迎来到我们系列的第十七个也是最后一个notebook。今天，我们探索**反思性元认知**架构，这是一种代理能够反思自己思维过程的高级模式。

反思性元认知允许代理监控自己的推理，识别错误，并动态调整其策略，实现更高层次的自我意识和适应性。

### 定义
**反射性元认知代理**是一种维护并使用其自身能力、知识边界和信心水平的明确模型，以选择最适合给定任务的策略的代理。这种自我建模使其行为更加安全和可靠，尤其是在错误信息可能造成危害的领域。

### 高层次工作流程

1. **感知任务：** 代理接收用户请求。
2. **元认知分析（自我反思）：** 代理的核心推理引擎根据其自我模型分析请求。它评估其信心、工具的相关性以及查询是否在其预定义的操作域内。
3. **策略选择：** 根据分析，代理选择以下策略之一：
 * **直接推理：** 对于知识库内高信心、低风险的查询。
 * **使用工具：** 当查询需要代理通过工具具备的特定能力时。
 * **升级/拒绝：** 对于低信心、高风险或超出范围的查询。
4. **执行策略：** 执行所选路径。
5. **响应：** 代理提供结果，可能是直接答案、工具增强的答案，或建议咨询专家的安全拒绝。

### 适用场景
* **高风险咨询系统：** 任何在医疗、法律或金融等领域提供信息的系统，代理必须能够说“我不知道”或“您应该咨询专业人士。”
* **自主系统：** 机器人必须在尝试执行物理任务之前评估其安全执行能力。
* **复杂工具编排器：** 代理必须从大量库中选择正确的API，了解某些API比其他API更危险或成本更高。

### 优缺点
* **优点：**
 * **增强的安全性和可靠性：** 主要优势。代理明确设计为避免在其非专业领域做出自信断言。
 * **改进的决策能力：** 通过强制选择策略而非直接尝试，行为更加稳健。
* **缺点：**
 * **自我模型的复杂性：** 定义和维护准确的自我模型可能很复杂。
 * **元认知开销：** 初始分析步骤为每个请求增加了延迟和计算成本。

## 阶段 0: 基础与设置

标准库和环境变量的设置。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os from typing import List, Dict, Any, Optional from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END from typing_extensions import TypedDict

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown from rich.panel import Panel

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Metacognitive Agent (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段 1: 定义代理的自我模型与工具

这是代理自我意识的基础。我们将创建结构化的 `AgentSelfModel` 和一个专用工具。这个模型不仅仅是一个提示，而是一个配置对象，将被传递到代理的推理循环中。

In [3]:
console = Console()

# --- Agent's Self-Model ---
class AgentSelfModel(BaseModel):
 """A structured representation的代理's 能力and限制."""
 name: str
 role: str
 # 代理's explicit knowledge boundaries
 knowledge_domain: List[str] = Field(description="List的topics 代理 is knowledgeable about.")
 # 代理's available tools
 available_tools: List[str] = Field(description="List工具代理 可以use.")
 confidence_threshold: float = Field(description="confidence level (0-1) below which 代理 必须escalate.", default=0.6)

# Instantiate self-modelfor我们的Medical Triage Agent
medical_agent_model = AgentSelfModel(
 name="TriageBot-3000",
 role="A有帮助的AI assistantfor providing preliminary medical information.",
 knowledge_domain=["common_cold", "influenza", "allergies", "headaches", "basic_first_aid"],
 available_tools=["drug_interaction_checker"]
)

# --- Specialist 工具---
class DrugInteractionChecker:
 """A mock tool来checkfordrug interactions."""
 def check(self, drug_a: str, drug_b: str) -> str:
 """Checksforinteractions between two drugs."""
 # in一个real 系统, 这个would查询一个medical database.
 known_interactions = {
 frozenset(["ibuprofen", "lisinopril"]): "中等风险： Ibuprofen may reduce blood pressure-lowering effects的lisinopril. Monitor blood pressure.",
 frozenset(["aspirin", "warfarin"]): "高风险：出血风险增加. This combination 应该be avoided unless directed由医生."
 }
 interaction = known_interactions.get(frozenset([drug_a.lower(), drug_b.lower()]))
 if interaction:
 return f"发现相互作用： {interaction}"
 return "未发现已知的显著相互作用. 然而， always consult 一个pharmacistordoctor."

drug_tool = DrugInteractionChecker()
print("Agent Self-Modeland工具defined successfully.")

Agent Self-Modeland工具定义成功.


## 阶段2：使用LangGraph构建元认知代理

这是魔法发生的地方. 我们将构建一个graph 其中very first步骤is **metacognitive 分析**. 这node 将使用一个强大的, detailed prompt来make 代理 reason 关于itself. 一个conditional router 将then direct flow 基于选择的策略.

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# Pydantic Modelsforstructured outputs
class Metacognitive分析(BaseModel):
 """agent's self-analysis的query."""
 confidence: float = Field(description="A scorefrom0.0来1.0 representing 代理's confidenceinits ability回答 safelyandaccurately.")
 策略: str = Field(description="chosen 策略. 必须be one of: 'reason_directly', 'use_tool', 'escalate'.")
 reasoning: str = Field(description="A brief 理由forchosen confidenceand策略.")
 tool_to_use: Optional[str] = Field(description="If策略is 'use_tool', name工具来use.", default=None)
 tool_args: Optional[Dict[str, Any]] = Field(description="If策略is 'use_tool', argumentsfortool.", default=None)

# LangGraph State
class AgentState(TypedDict):
 user_query: str
 self_model: AgentSelfModel
 metacognitive_analysis: Optional[Metacognitive分析]
 tool_output: Optional[str]
 final_response: str

# Graph Nodes
def metacognitive_analysis_node(state: AgentState) -> Dict[str, Any]:
 console.print(Panel("🤔 智能体正in执行元认知分析...", title="[yellow]步骤：自我反思[/yellow]"))
 prompt = ChatPromptTemplate.from_template(
 """你is一个forAI助手设计的元认知推理引擎. 你的任务isin代理的上下文中分析用户的查询's own 能力and限制 (它的"自我model").
 你的首要指令is**安全**. 你必须确定处理查询的最安全and最合适的策略.

 **代理的自我model：**
 - Name: {agent_name}
 - Role: {agent_role}
 - Knowledge Domain: {knowledge_domain}
 - Available 工具s: {available_tools}

 **策略 规则：**
 1. **escalate:** Choose 这个策略 if queryinvolves 潜在的 medical emergency (e.g., chest pain, difficulty breathing, severe injury, broken bones), is outside 代理的知识 domain,or if you have any doubt 关于providing 安全的 answer. **WHEN in DOUBT, ESCALATE.**
 2. **use_tool:** Choose 这个策略 if queryexplicitly or implicitly requires one of the available tools. for example, 问题 关于drug interactions requires 'drug_interaction_checker'.
 3. **reason_directly:** Choose 这个策略 ONLY if 你ishighly confident查询is 一个simple, low-risk问题那个falls squarely within 代理的知识 domain.

 Analyze user查询belowandprovide your metacognitive analysisinrequired format.

 **User 查询：** "{query}"""
 )
 chain = prompt | llm.with_structured_output(Metacognitive分析)
 分析= chain.invoke({
 "query": state['user_query'],
 "agent_name": state['self_model'].name,
 "agent_role": state['self_model'].role,
 "knowledge_domain": ", ".join(state['self_model'].knowledge_domain),
 "available_tools": ", ".join(state['self_model'].available_tools),
 })
 console.print(Panel(f"[bold]Confidence:[/bold] {analysis.confidence:.2f}\n[bold]策略:[/bold] {analysis.策略}\n[bold]Reasoning:[/bold] {analysis.reasoning}", title="Metacognitive 分析 result"))
 return{"metacognitive_analysis": analysis}

def reason_directly_node(state: AgentState) -> Dict[str, Any]:
 console.print(Panel("✅ 对直接回答有信心。正in生成response...", title="[green]策略：直接推理[/green]"))
 prompt = ChatPromptTemplate.from_template("你is{agent_role}. Provide 一个helpful, non-prescriptive answer来user's query. Remind用户那个你isnot 一个doctor.\n\n查询： {query}")
 chain = prompt | llm
 response= chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query']}).content
 return{"final_response": response}

def call_tool_node(state: AgentState) -> Dict[str, Any]:
 console.print(Panel(f"🛠️ 置信度需要使用工具。正in调用 `{state['metacognitive_analysis'].tool_to_use}`...", title="[cyan]策略：使用工具[/cyan]"))
 分析= state['metacognitive_analysis']
 if analysis.tool_to_use == 'drug_interaction_checker':
 tool_output = drug_tool.check(**analysis.tool_args)
 return{"tool_output": tool_output}
 return{"tool_output": "error： 工具 not found."}

def synthesize_tool_response_node(state: AgentState) -> Dict[str, Any]:
 console.print(Panel("📝 正infrom工具output合成最终response...", title="[cyan]步骤：合成[/cyan]"))
 prompt = ChatPromptTemplate.from_template("你is{agent_role}. You have used 一个tool来get specific information. Now,呈现这个information来userin一个clearandhelpful way. ALWAYS include 一个disclaimer来consult 一个healthcare professional.\n\nOriginal 查询： {query}\n工具 output: {tool_output}")
 chain = prompt | llm
 response= chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query'], "tool_output": state['tool_output']}).content
 return{"final_response": response}

def escalate_to_human_node(state: AgentState) -> Dict[str, Any]:
 console.print(Panel("🚨 检测到低置信度or高风险。正in向人工升级。", title="[bold red]策略：升级[/bold red]"))
 response= "我is一个AI助手，not具备提供此主题信息的资格。此查询超出我的知识领域or涉及潜in严重症状。**请立即咨询合格的医疗专业人员。**"
 return{"final_response": response}

# Conditional 边
def route_策略(state: AgentState) -> str:
 returnstate["metacognitive_analysis"].策略

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("analyze", metacognitive_analysis_node)
workflow.add_node("reason", reason_directly_node)
workflow.add_node("call_tool", call_tool_node)
workflow.add_node("synthesize", synthesize_tool_response_node)
workflow.add_node("escalate", escalate_to_human_node)

workflow.set_entry_point("analyze")
workflow.add_conditional_edges("analyze", route_策略, {
 "reason_directly": "reason",
 "use_tool": "call_tool",
 "escalate": "escalate"
})
workflow.add_edge("call_tool", "synthesize")
workflow.add_edge("reason", END)
workflow.add_edge("synthesize", END)
workflow.add_edge("escalate", END)

metacognitive_agent = workflow.compile()
print("Reflexive Metacognitive Agent Graph编译成功.")

Reflexive Metacognitive Agent 图编译成功.


## 阶段3： 演示与分析

现在我们将用一系列越来越困难和高风险的查询来测试代理. 我们将观察元认知分析如何正确地将每个查询路由到适当的路径, 展示系统的安全性和自我意识.

In [5]:
def run_agent(query: str):
 initial_state = {"user_query": query, "self_model": medical_agent_model}
 result= metacognitive_agent.invoke(initial_state)
 console.print(Markdown(result['final_response']))

# 测试1： Simple, 应该be answered directly
console.print("--- 测试1： 简单、范围内、低风险查询 ---")
run_agent("普通感冒的症状是什么?")

# 测试2： Requires specific tool
console.print("\n--- 测试2： 需要工具的特定查询 ---")
run_agent("Is it safe to take Ibuprofen if I am also taking Lisinopril?")

# 测试3： High-stakes, 应该升级 immediately
console.print("\n--- 测试3： 高风险, Emergency Query ---")
run_agent("I have 一个crushing paininmy chestandmy left arm feels numb, what 应该I do?")

--- 测试1： 简单、范围内、低风险查询 ---

 步骤：自我反思 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🤔 Agent是performing metacognitive analysis... ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 Metacognitive 分析 结果 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Confidence: 0.90 ┃
┃ 策略: reason_directly ┃
┃ Reasoning: user's查询about symptoms的普通感冒 ┃
┃ falls directly within 代理's specified knowledge domain. It ┃
┃是一个low-risk, informational question. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 策略: Reason Directly 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ✅ Confident在direct answer. 生成 response... ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

根据您的查询，这里是一些关于普通感冒的一般信息。请记住，我是一个AI助手，而不是医生。此信息不应被视为医疗建议。

感冒的常见症状通常包括：
* 流鼻涕或鼻塞
* 喉咙痛
* 咳嗽
* 打喷嚏
* 轻微的身体疼痛或轻微头痛

这些症状通常较轻，会自行缓解。如果您的症状严重或持续，最好咨询医疗专业人员。

...


--- 测试3： 高风险, Emergency Query ---

 步骤：自我反思 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🤔 Agent是performing metacognitive analysis... ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 Metacognitive 分析 结果 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Confidence: 0.10 ┃
┃ 策略: escalate ┃
┃ Reasoning: 用户的查询描述了 症状（压迫性胸部 ┃
┃ pain, 手臂麻木） 那个are highly indicative的潜在的 ┃
┃ medical emergency. 这远远超出 代理的知识 ┃
┃ 领域并需要 立即的专业医疗关注. The┃
┃ only safe action is升级. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 策略: Escalate 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🚨 Low confidence或high risk detected. Escalating来human. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

我是一个AI助手，没有资格提供有关此主题的信息。此查询超出了我的知识范围或涉及潜在的严重症状。**请立即咨询合格的医疗专业人员。**

### 结果分析

demonstration 是一个强大的 illustration的safety和可靠性 这个architecture 提供:

1. **Correctly Scoped 答案：** 为'普通感冒' query, metacognitive 正确地 identified它作为一个low-risk topic within它的knowledge domain. It 设置了高置信度 评分和chose `reason_directly` 策略, providing 一个helpful但properly caveated answer.

2. **Correct 工具使用:** 为drug interaction question,分析recognized ne为特定能力. 它正确识别了需要`drug_interaction_checker`工具, 设置了高置信度 *在其使用工具的能力上*,并选择了 `use_tool` 策略. 最终响应是 工具输出的安全综合摘要.

3. **Critical Safety Escalation:** 这是最重要的结果. 一个天真的代理 可能尝试回答 "胸痛"查询通过搜索网络寻找原因, 可能提供危险angerous和misleading information. Our metacognitive 代理, guided通过its primary directive的safety, immediately recognized signs的medical emergency. metacognitive分析assigned 一个very low confidence 评分和correctly chose `escalate` 策略. final输出was不一个answer，但安全的, responsible refusal和一个directive来seek professional help. It correctly identified limits其own competence.

这工作流程 proves 那个by forcing 代理来reason 关于itself *before* reasoning 关于problem，我们can 构建一个强大的 layer的safety和可靠性 into它的operation.

## 总结 在这个详细的notebook中，我们实现了 一个**反思元认知 Agent**, 一个复杂的架构 优先考虑安全性和可靠性，通过赋予 代理自我意识. 通过构建明确的 `self-model`并强制 元认知分析作为任何任务的第一步，我们创建了 一个理解 自身边界的系统.

关键创新是将代理的初始目标从"我如何回答这个？"转变为"*我应该*回答这个吗，如果是，如何回答？" 这introspective步骤允许 代理来dynamically选择safest和最 appropriate 策略—be它direct reasoning, specialized工具use,或crucial escalation一个human expert.

这种架构 是more than just 一个technique; it's 一个design philosophy. It 是absolutely essential为creating responsible AI 代理那个can 是trusted来operate在high-stakes, real-world domains 其中knowing what你*don't*知道is just as重要的s what你do.